# IQPE and evolution of VQE output state

This notebook demonstrates using an output result state from [VQE](https://qiskit.org/documentation/stubs/qiskit.aqua.algorithms.VQE.html) as the input, starting state for evolution by the [IQPE](https://qiskit.org/documentation/stubs/qiskit.aqua.algorithms.IQPE.html) (Iterative Quantum Phase Estimation) algorithm. 

This is accomplished by first running VQE and then preparing IQPE's initial state using the variational form as produced by VQE upon its termination.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, IQPE
from qiskit.aqua.operators import I, X, Z
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.initial_states.var_form_based import VarFormBased

First we create a qubit operator for VQE. Here we will use the same operator as used in the algorithms introduction, which was originally computed by Qiskit Chemistry for an H2 molecule.

In [2]:
H2_op = (-1.052373245772859 * I ^ I) + \
        (0.39793742484318045 * I ^ Z) + \
        (-0.39793742484318045 * Z ^ I) + \
        (-0.01128010425623538 * Z ^ Z) + \
        (0.18093119978423156 * X ^ X)

First we will use the classical NumPyMinimumEigensolver to compute a reference ground state energy value so later we can compare with the VQE result first, then IQPE's.

In [3]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=H2_op)
ref_value = result.eigenvalue.real
print(f'Reference value: {ref_value:.5f}')

Reference value: -1.85728


Having established the reference ground energy, we next carry on with our experiment. First we configure a VQE algorithm instance. The idea is that we can set an termination condition such that the VQE instance returns rather quickly with a rough estimation result.

In [4]:
seed = 5
aqua_globals.random_seed = seed

var_form = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz', reps=3)
spsa = SPSA(maxiter=10)
backend = BasicAer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

vqe = VQE(var_form=var_form, optimizer=spsa, quantum_instance=qi)
result_vqe = vqe.compute_minimum_eigenvalue(operator=H2_op)

print(f'VQE estimated the ground energy to be {result_vqe.eigenvalue.real:.5f}')
print(f'Delta from reference energy value is {(result_vqe.eigenvalue.real - ref_value):.5f}')

VQE estimated the ground energy to be -1.74063
Delta from reference energy value is 0.11664


As previously indicated, the energy estimation result is rather rough--it is far from being an acceptable final estimation figure. But, it is close enough such that the accompanying variational form might be a reasonably good approximation to the ground eigenstate, which means the corresponding wave function can serve as the initial state for the IQPE execution that follows. We next prepare such an initial state.

In [5]:
vqe_state = VarFormBased(var_form, result_vqe.optimal_parameters)

With the VQE-generated quantum state wave function prepared for the initial state, we now go ahead with configuring and running an IQPE instance.

In [6]:
qi = QuantumInstance(backend, shots=100, seed_simulator=seed, seed_transpiler=seed)

iqpe = IQPE(state_in=vqe_state, num_time_slices=1, num_iterations=6,
            expansion_mode='suzuki', expansion_order=2, quantum_instance=qi)
result_iqpe = result_vqe = iqpe.compute_minimum_eigenvalue(operator=H2_op)

print(f"Continuing with VQE's result, IQPE estimated the ground energy to be {result_iqpe.eigenvalue.real:.5f}")
print(f'Delta from reference energy value is {(result_iqpe.eigenvalue.real - ref_value):.5f}')

Continuing with VQE's result, IQPE estimated the ground energy to be -1.84917
Delta from reference energy value is 0.00811


As can be seen, the resultant ground state energy estimation, as produced by IQPE, is much more accurate that the intermediate result produced by VQE.

In [7]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright